# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [6]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [7]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [8]:
browser = webdriver.Chrome(executable_path='../chromedriver')

browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

while True:

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

#     提示：可參考以下的Stack Overflow: 
#     https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [10]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [11]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 定存族哭哭！銀行台幣定儲利率跌破1%　每百萬存款1年利息少2523元

▲台銀總行。（圖／記者吳靜君攝）
記者吳靜君／台北報導
中央銀行自今（20）日起調降重貼現率、擔保放款融通利率及短期融通利率各1碼（0.25）個百分點。台銀、土銀為配合中央銀行貨幣政策，並反映當前市場利率走勢，經衡酌市場競爭及資金狀況，將自下周一（23日）起，調整利率，一年期定期儲蓄存款機動利率從1.09%、固定利率1.07%，分別調降到0.84%和0.79%。
以存款100萬元、整存整付、複利息來計算，原本一年的利息為10955元，調降後一年的利息僅剩8432元，一年減少2523元。
台銀表示，同時也調降新台幣各天期一般存款及500萬元以上定期存款及定期儲蓄存款牌告利率，調整幅0.06到0.30個百分點，並調降活期性存款利率，調整幅度0.01到0.10個百分點。土銀也表示，「一般」定期性存款各天期機動利率均調降，調降幅度0.25個百分點到0.295個百分點，同樣是下周一起調整。
▼台銀存款新牌告利率。（圖／台銀提供）
----------------------------------------------------------------------

[1] 台指期盤中大漲810點　分析師：逆價差縮小、盤勢止穩反彈機會大

▲台指期3月20日走勢。（圖／取自期交所）
記者余弦妙／台北報導
國安基金宣示進場護盤有效，台指期今（20）日開盤受到激勵大漲206點，來到8562點，接著持續上揚，盤中甚至突破9100點，來到今日高點9,166點，大漲810點大幅縮小逆價差，尾盤雖有小幅拉回，不過終仍以9013點作收，上漲657點，與現貨相較，逆價差221.09點。展望後市，萬寶投顧執行長王榮旭分析，市場出現止穩甚至扭轉，建議投資人不要做空，應該偏多操作。
王榮旭表示，今天外資在期現貨都進行調節，但調節力道也減輕很多，外資賣219億台股還能夠上漲，這也意味著國安基金進場有效，因此在外資想做空做不了的情況之下，不排除下周有可能會反過來做多，且今天的逆價差也大幅縮小。
王榮旭說，外資現在賣也是因應美國基金的贖回，因此也是被動式的賣超台股，且今天亞洲股市

全台麵粉供應龍頭企業聯華實業今（20）日宣佈，將與鼎泰豐麵粉供應商僑泰興簽訂合作意向書，規劃投資聯華製粉公司股權，以強化麵粉事業營運規模與生產效率，強強聯手可望提升市佔率至近三成，未來不排除規劃上市。
聯華實業投控目前為全台產量最大的麵粉廠，年銷售量超過2億公斤。它是繼南僑後，第二家轉型為控股公司的食品公司，市場解讀此舉有利營運彈性，集團亦加速旗下子公司上市櫃規劃。
全台麵粉供應龍頭企業聯華實業去年九月轉型為控股公司，旗下事業分割為麵食、租賃及電子事業，總經理林信宏指出，麵食事業透過分割成立聯華製粉食品公司，董事會並通過與僑泰興進行策略合作可行性討論。
林信宏指出，聯華實業轉型為投資投股公司後，集團所跨足之各領域事業將採各自獨立發展政策，以使各事業體可專注深耕於其領域，與策略合作夥伴進行業務合作、合資或參股等更具靈活發展。
僑泰興於1953年成立，以嘉禾牌麵粉聞名餐飲業界，為國內知名老店如鼎泰豐等指定使用之麵粉品牌，為國內第一家獲得SQF食品安全最高等級驗證之專業麵粉廠，主要麵粉產品有「藍菊花牌」、「劍蘭牌」、「黃僑牌」等品牌。
----------------------------------------------------------------------

[14] 衛生紙原物料供貨一切正常　業者對瘋搶行為「無法理解」！

▲連鎖超市衛生紙被搶空。（圖／資料照）
記者余弦妙／台北報導
新冠肺炎疫情持續增溫，台灣確診案例已經突破百例，引起不少民眾恐慌，趕著去搶購物資，其中衛生紙更是成為大多數民眾搶購目標，甚至有賣場為確保消費者購物權益，祭出限購規則。對此，國內衛生紙廠商對於搶購行為表示「不明所以」。
正隆表示，目前衛生紙生產狀態都是正常，原物料的庫存量也都足夠，可以說面對民眾的搶購潮完全可以應付得來，只是目前就算生產的出來，但在物流配送方面遠遠趕不及民眾搶購速度，因此在生產端與消費端就出現的時間差，進而發生賣場架上沒有衛生紙的狀況。
另外，永豐餘說，目前公司生產狀態都一如既往，並沒有任何改變，也無出現衛生紙供應不足，或是原物不足的情況。對於現在市場的搶購情況則表示，這應該只是恐慌情緒所引發的，並強調公司在衛生紙的供貨絕對沒有問題，運作一切正常無須緊張。
目前國內三大紙廠分別為金百利（旗下品牌舒潔、可麗舒、可利雅）、永豐餘（五月花、柔情、得意）、正隆（春風

▲口罩國家隊陸續提升產能，助攻口罩日均產能達到1000萬片以上。（圖／記者湯興漢攝）
記者林淑慧／台北報導
口罩國家隊日夜趕工「你累了嗎？」，有藥師在粉專發文，指出近日發現口罩瑕疵逐漸增加，經濟部今（20）日回應表示，已提醒藥局、超商等通路，瑕疵口罩先不出售，網路預購的民眾也能到超商換貨，日後會再統一回收。
有藥師在《北大藥局》粉專PO文指出，近幾天發現口罩瑕疵有增加趨勢，除了鋼條脫落、繩子貼反、還有耳掛線斷及出現髒汙的情況，藥師無奈地說，直接少掉八包的量，「瑕疵品會影響到每天實際能發放的數量，請大家多包容」。
對此，經濟部工業局官員指出，工業局在每一個口罩工廠均有駐廠人員，已經要求駐廠同仁轉達口罩生產者，應注意品質的管控，也謝謝藥房藥師在第一缐的辛苦。
另外，由經濟部斥資加碼的第二批32條新產線，原本表訂將在三月底組裝完成，但在口罩國家隊熱血投入超過2000人次趕工下，今日可望提前完工，並推升口罩日均產能上看1200萬片。
----------------------------------------------------------------------

[24] 台股強勢攻上9200點！大立光、聯發科攻漲停　指數暴衝570點

▲ 中央銀行、金管會及國安基金救市三支箭齊發，台股今日一掃陰霾應聲大漲，盤中漲點570點，收復9200點關卡。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台股今（20）日開盤後一路飆升，連續收復9000、9100與9200點關卡，漲點擴大到500點以上，目前來到9251點、上漲570點。
股王大立光（3008）強勢攻漲停，目前來到3575元；聯發科（2454）更是自10點左右就在漲停板附近狹幅震盪，目前也維持漲停價位，為301元。
另外，台積電（2330）目前亦強漲8.67%、來到270元；鴻海（2317）也收復70元大關，目前在70.9元、上漲6.49%
看到漲跌家數，目前上市總共有127家亮燈漲停，763家維持紅盤；28家下跌，跌停則為3家。
----------------------------------------------------------------------

[25] 國際原油期貨今天續漲！西德州上漲4.25%　布蘭特上漲2.14%

▲國際原油價格上漲。（圖／路透）
記者陳心怡／綜合報導
面對沙烏

▲東森購物捐贈防疫物資，向辛苦的警消醫護、疫苗研發團隊與其家人致敬。（圖／東森購物提供）
記者洪菱鞠／台北報導
為了向守護全民健康的「台灣隊長」包括醫護警消、疫苗研發團隊與其家人致敬，東森購物電話行銷團隊精選目前通路最暢銷的保健品和保養品，邀請供應商共襄盛舉，不計成本全數捐贈予第一線醫護人員，希望能為他們及家人補充健康好體力、維持美麗好氣色，而此次贊助物資總金額高達新台幣1,216萬元。
東森購物表示，此次捐贈防疫物資包括東森自然美贊助的「自然美－瑞昇超逆齡賦活膠囊」及「東森自然美瑞昇超逆齡賦活膠囊」，東森自然美執行長林燕玲觀察到，女性醫護人員因長時間戴口罩，導致臉部肌膚變得容易乾癢、脫皮或泛紅，希望她們在辛苦工作之餘，也能收到讓肌膚保持最佳狀況的優質MIT保養產品，自然散發美麗好氣色。
在新冠肺炎疫情影響下，民眾降低出門頻率，電商平台此時成為購物安全考量首選，東森購物電話行銷執行長紀坤傑指出，從疫情爆發開始，抗疫產品和保健食品等需求都非常大。東森購物始終秉持顧客至上的服務精神，除持續關心疫情現況，也積極配合政府政策，疫情期間因國內農漁產外銷受到衝擊，因此率先響應農委會協助開賣鳳梨釋迦、午仔魚與石斑魚力挺小農，另外也攜手新北市政府推出「新北專區」，幫業者拓展虛擬通路商機，更不斷精進服務品質，為台灣社會盡一份心力，也持續為所有會員帶來最愉悅舒適的購物體驗。
▲▼疫情期間，東森購物率先響應農委會政策，於旗下電視購物與購物網等通路大力促銷國內優質農漁產品，台東鳳梨釋迦一開賣立即狂銷3萬台斤，午仔魚與石斑魚2小時熱賣8,000條。（上圖／記者徐文彬攝、下圖／東森購物提供）
----------------------------------------------------------------------

[37] 陸84艘輪船赴沙國搶原油　網嘆：像雙11...真相出爐了

▲國際油價持續下跌。（圖／路透社）
記者曾俊豪／綜合報導
大陸網站近日流傳一篇文章，表示有84艘中國輪船啟航，前往沙國購買原油。不過，文章經媒體查證，並非事實。業內人士分析，國際油價暴跌之時，大陸煉廠從海外進口原油，理論上有利於降低原油進口成本。但加大進口量仍有其他因素限制，搶油對煉廠的提振作用並不會很快顯現。《澎湃》新聞報導，這篇以《中國84艘巨輪拔錨起航 全球最大船隊直撲海灣 外媒感嘆實力懸